# Linear Function

In [5]:
import torch
import torch.nn as nn
import numpy as np

# Linear Function
# input dim = 5, output dim = 1
linear = nn.Linear(5, 1)
x = torch.FloatTensor([-2, -1, 0, 1, 2])
print(linear(x)) 
print(linear.parameters())
print([x for x in linear.parameters()])
print(linear.bias)

tensor([-0.5504], grad_fn=<AddBackward0>)
<generator object Module.parameters at 0x127110318>
[Parameter containing:
tensor([[ 0.4346,  0.1119, -0.0582, -0.4244,  0.2762]], requires_grad=True), Parameter containing:
tensor([0.3027], requires_grad=True)]
Parameter containing:
tensor([0.3027], requires_grad=True)


# Activation function

In [2]:
x = torch.FloatTensor([-2, -1, 0, 1, 2])
print(x)

# ReLU
relu = nn.ReLU()
print(relu(x))  # max(0, input)

# Sigmoid
sigmoid = nn.Sigmoid() # 고정된 함수에 넣는 거. 
print(sigmoid(x))

# Tanh
tanh = nn.Tanh()
print(tanh(x))

# Softmax
softmax = nn.Softmax(dim = 0)
print(softmax(x))

# softmax와 sigmoid는 다르다.



tensor([-2., -1.,  0.,  1.,  2.])
tensor([0., 0., 0., 1., 2.])
tensor([0.1192, 0.2689, 0.5000, 0.7311, 0.8808])
tensor([-0.9640, -0.7616,  0.0000,  0.7616,  0.9640])
tensor([0.0117, 0.0317, 0.0861, 0.2341, 0.6364])


# MSE Loss

In [3]:
a = torch.FloatTensor([1, 4, 2])
b = torch.FloatTensor([1, 3, 2])

# MSE Loss
mse = nn.MSELoss()
print(mse(a, b))

tensor(0.3333)


In [4]:
a = torch.FloatTensor([[1, 4, 2], [0.5, 0.3, 0.2]])
b = torch.LongTensor([0, 0]) # label값.

cross_entropy = nn.CrossEntropyLoss()
print(cross_entropy(a,b)) # 학습할 x값을 a 위치, 정답을 b 위치.=

tensor(2.0548)


# Convolutional Network

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size = 32
learning_rate = 0.01
num_epochs = 5

# MNIST Dataset DataLoader
train_dataset = datasets.MNIST(root='./data', train=True,
                        transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False,
                        transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2), #kernal size = filter size. 16은 필터 개수와 동일.
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [7]:
# Model
model = CNN()

# Loss
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(num_epochs):
    for i, (img, label) in enumerate(train_loader, 1):
        img, label = Variable(img), Variable(label)
        out = model(img)
        loss = criterion(out, label)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/5], Step [100/1875], Loss: 0.3772
Epoch [1/5], Step [200/1875], Loss: 0.2697
Epoch [1/5], Step [300/1875], Loss: 0.1680
Epoch [1/5], Step [400/1875], Loss: 0.2900
Epoch [1/5], Step [500/1875], Loss: 0.1734
Epoch [1/5], Step [600/1875], Loss: 0.2310
Epoch [1/5], Step [700/1875], Loss: 0.1478
Epoch [1/5], Step [800/1875], Loss: 0.0703
Epoch [1/5], Step [900/1875], Loss: 0.0317
Epoch [1/5], Step [1000/1875], Loss: 0.0087
Epoch [1/5], Step [1100/1875], Loss: 0.1166
Epoch [1/5], Step [1200/1875], Loss: 0.0799
Epoch [1/5], Step [1300/1875], Loss: 0.0158
Epoch [1/5], Step [1400/1875], Loss: 0.0952
Epoch [1/5], Step [1500/1875], Loss: 0.0652
Epoch [1/5], Step [1600/1875], Loss: 0.0604
Epoch [1/5], Step [1700/1875], Loss: 0.0390
Epoch [1/5], Step [1800/1875], Loss: 0.0298
Epoch [2/5], Step [100/1875], Loss: 0.3216
Epoch [2/5], Step [200/1875], Loss: 0.1071
Epoch [2/5], Step [300/1875], Loss: 0.1038
Epoch [2/5], Step [400/1875], Loss: 0.0083
Epoch [2/5], Step [500/1875], Loss: 0.0220
Ep

In [9]:
# Test the model
model.eval()

correct = 0
total = 0
for img, label in test_loader:
    out = model(img)
    _, predicted = torch.max(out.data, 1)
    total += label.size(0)
    correct += (predicted == label).sum().item()
    
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model_nn.ckpt')

Test Accuracy of the model on the 10000 test images: 98.78 %


# 과제1. Neural Network에 활성함수 / batchNorm 추가

In [6]:
import torch.nn.functional as F

In [7]:
# toTensor() => dataloader에서 사용함. 이미지 값을 tensor로 변경해 준다. (type 변경.)

In [11]:
class Neuralnetwork(nn.Module):
    def __init__(self, num_classes = 10):
        super(Neuralnetwork, self).__init__()
        self.layer1 = nn.Linear(28*28, 100)
        self.bn1 = nn.BatchNorm1d(100)
        self.layer2 = nn.Linear(100,200)
        self.bn2 = nn.BatchNorm1d(200)
        self.layer3 = nn.Linear(200, num_classes)
        
    def forward(self, x):
        out = x.view(x.size(0),-1) # x.size는 batch size다. 
        out = F.relu(self.bn1(self.layer1(out)))
        out = F.relu(self.bn2(self.layer2(out)))
        out = F.softmax(self.layer3(out), dim = 1) # dim = 1 의미는?
        return out

In [12]:
model = Neuralnetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
for epoch in range(num_epochs):
    for idx, (img, label) in enumerate(train_loader,1):
        img, label = Variable(img), Variable(label)
        out = model(img)
        loss = criterion(out,label)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (idx+1) % 100 ==0:
            print("Epoch [{}/{}], Step [{}/{}], Loss : {:.4f}".format(epoch+1, num_epochs, idx+1, len(train_loader), loss.item()))
            

Epoch [1/5], Step [100/1875], Loss : 2.2822
Epoch [1/5], Step [200/1875], Loss : 2.2484
Epoch [1/5], Step [300/1875], Loss : 2.1661
Epoch [1/5], Step [400/1875], Loss : 2.1070
Epoch [1/5], Step [500/1875], Loss : 2.1090
Epoch [1/5], Step [600/1875], Loss : 2.1622
Epoch [1/5], Step [700/1875], Loss : 1.9907
Epoch [1/5], Step [800/1875], Loss : 1.9820
Epoch [1/5], Step [900/1875], Loss : 1.9085
Epoch [1/5], Step [1000/1875], Loss : 1.8737
Epoch [1/5], Step [1100/1875], Loss : 1.9353
Epoch [1/5], Step [1200/1875], Loss : 1.9701
Epoch [1/5], Step [1300/1875], Loss : 1.8549
Epoch [1/5], Step [1400/1875], Loss : 1.9177
Epoch [1/5], Step [1500/1875], Loss : 1.7357
Epoch [1/5], Step [1600/1875], Loss : 1.7155
Epoch [1/5], Step [1700/1875], Loss : 1.7786
Epoch [1/5], Step [1800/1875], Loss : 1.8340
Epoch [2/5], Step [100/1875], Loss : 1.6318
Epoch [2/5], Step [200/1875], Loss : 1.7387
Epoch [2/5], Step [300/1875], Loss : 1.7689
Epoch [2/5], Step [400/1875], Loss : 1.7520
Epoch [2/5], Step [500/

In [13]:
model.eval()

correct = 0
total = 0
for img, label in test_loader:
    out = model(img)
    _, predicted = torch.max(out.data,1)
    total += label.size(0)
    correct+= (predicted == label).sum().item()


print('Test Accuracy of the Model on the 10000 test images : {} %'.format(100 * correct / total))

Test Accuracy of the Model on the 10000 test images : 95.13 %


# 과제 : CIFAR - 10에 CNN 구현

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size = 4
learning_rate = 0.01
num_epochs = 5

# CIFAR-10 Dataset DataLoader
trainset = datasets.CIFAR10(root='./data', train=True, download=True,
                                        transform=transforms.ToTensor())

testset = datasets.CIFAR10(root='./data', train=False, download=True, 
                                       transform=transforms.ToTensor())


train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, 5), # kernal size = filter size. 16은 필터 개수와 동일. 여기서는 5만 넣어도 5*5로 인식함.
            # output channel은 임의로 바꿔 줘도 된다. 다만 그 다음 conv2d에서 input 맞춰줘야 함)
            
            #nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0), # kernal size = filter size. 16은 필터 개수와 동일.
#             nn.BatchNorm2d(6),
            # RELU의 기준점은 0. 전체적으로 -값을 batchnorm을 통해 정규분포화하는 것. 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
#             nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(-1, 16 * 5 * 5) #
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        
        return out

In [10]:
# Model
model = CNN()

# Loss
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

In [11]:
# 오류를 보니 conv2d의 input, output layer 설정에 오류가 있는 것 같은데, 어디를 어떻게 수정해야 할 지 모르겠습니다.

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):

        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 2.252
[1,  4000] loss: 2.041
[1,  6000] loss: 1.894
[1,  8000] loss: 1.739
[1, 10000] loss: 1.638
[1, 12000] loss: 1.567
[2,  2000] loss: 1.503
[2,  4000] loss: 1.474
[2,  6000] loss: 1.460
[2,  8000] loss: 1.423
[2, 10000] loss: 1.407
[2, 12000] loss: 1.394
[3,  2000] loss: 1.351
[3,  4000] loss: 1.334
[3,  6000] loss: 1.305
[3,  8000] loss: 1.313
[3, 10000] loss: 1.280
[3, 12000] loss: 1.273
[4,  2000] loss: 1.224
[4,  4000] loss: 1.209
[4,  6000] loss: 1.212
[4,  8000] loss: 1.220
[4, 10000] loss: 1.202
[4, 12000] loss: 1.197
[5,  2000] loss: 1.154
[5,  4000] loss: 1.129
[5,  6000] loss: 1.136
[5,  8000] loss: 1.137
[5, 10000] loss: 1.139
[5, 12000] loss: 1.140


In [12]:
# Test the model
model.eval()

correct = 0
total = 0
for img, label in test_loader:
    out = model(img)
    _, predicted = torch.max(out.data, 1)
    total += label.size(0)
    correct += (predicted == label).sum().item()
    
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model_nn_cifar10.ckpt')

Test Accuracy of the model on the 10000 test images: 58.41 %
